In [ ]:
import mne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from ev_parser import gesture_parser
from functools import partial
from scipy import interpolate
from sklearn.preprocessing import StandardScaler
%matplotlib inline

path_out = 'C:/skoltech_hand_writing'
subj = ''

In [ ]:
### SCRIPT "3_Combine_all.ipynb"
for code in np.arange(1, 6):
    # read processed files with marks and tragectories
    trials=pd.read_csv(f'{path_out}/hand_writing/{subj}/preprocessed/Rec_{code}/trials_processed.csv',sep=';')
    trials_=trials[['ts','X','Y','Epoch']]
    trials_=trials_[trials_['Epoch']==1]
    times=trials_.ts.values
    inds_from_start=np.where(times>=0)[0]
    times=times[inds_from_start]

    marks=pd.read_csv(f'{path_out}/hand_writing/{subj}/preprocessed/Rec_{code}/marks_processed.csv',sep=';')
    dfdf=pd.DataFrame(columns=trials.columns)

    n_trials=50
    # remove excessive epochs
    trials=trials[trials['Epoch']>0]
    # add everyting ina single pd dataframe and find events that correspond to trial start and finish
    for i in range(1,n_trials):
        
        tt=tt=trials[trials['Epoch']==i].iloc[0].values[2]-1/1000
        dfdf.loc[len(dfdf)]=[None,'/dev/input/event0:',tt,None,-2,-2,i-1]
        
        tt=tt=trials[trials['Epoch']==i].iloc[0].values[2]-2/1000
        dfdf.loc[len(dfdf)]=[None,'/dev/input/event0:',tt,None,-2,-2,i-1]

        tt=tt=trials[trials['Epoch']==i].iloc[-1].values[2]+1/1000
        dfdf.loc[len(dfdf)]=[None,'/dev/input/event0:',tt,None,-3,-3,i]
        
        tt=tt=trials[trials['Epoch']==i].iloc[-1].values[2]+2/1000
        dfdf.loc[len(dfdf)]=[None,'/dev/input/event0:',tt,None,-3,-3,i]

    marks = marks[['Unnamed: 0','INPUT','ts','ResetCounter','X','Y','Epoch']]
    df_all=pd.concat([marks,trials,dfdf])
    df_all.sort_values('ts',inplace=True)

    # reading events:
    events=np.load(f'{path_out}/hand_writing/{subj}/preprocessed/Rec_{code}/events.npy')
    
    # ids of the first and the last trials of each session:::
    code_ids = {1 : [1,51], 2: [52,102], 3 : [103,153], 4: [154,204], 5: [205,255]}
    events=events[code_ids[code][0]:code_ids[code][-1]]
    raw=mne.io.read_raw_fif(f'{path_out}/hand_writing/{subj}/preprocessed/Rec_1/first_raw.fif', preload=True)
    time_start=events[0,0]//raw.info['sfreq']
    epochs=mne.Epochs(raw,events=events, event_id=0, tmin=0,tmax=8,preload=True, baseline=None,
                    reject_by_annotation=False)
    times=df_all.ts.values
    # extract tragectories and interpolate to account for the difference in sampling points
    x=df_all.X.values
    y=df_all.Y.values
    sf=raw.info['sfreq']
    f = interpolate.interp1d(times, x)
    timesnew = np.arange(times[0], times[-1], 1/sf)
    xnew = f(timesnew)  

    f = interpolate.interp1d(times, y)
    timesnew = np.arange(times[0], times[-1], 1/sf)
    ynew = f(timesnew)
    adding_in_secs=np.round(events[0,0]/epochs.info['sfreq'],3)
    adding_in_ts=events[0,0]
    marks_ch='B'
    sensor_ch='C'

    # PLOT IN SEPARATE CELLS TO PREVENT PYTHON CRUSHING!!!
    # =======================================================================
    # plt.plot(times-times[0]+adding_in_secs+58, x,'.-b', alpha=0.4)
    # plt.scatter(times[np.where(x==-1)[0]]-times[0]+adding_in_secs, xnew[np.where(x==-1)[0]], color = 'black', label = 'erase')

    # #plt.scatter(timesnew[np.where(xnew==-1)[0]]-timesnew[0]+adding_in_secs, xnew[np.where(xnew==-1)[0]])
    # plt.scatter(timesnew[np.where(xnew==-2)[0]]-timesnew[0]+adding_in_secs, xnew[np.where(xnew==-2)[0]], color = 'blue', label = 'draw start')


    # plt.plot(raw[marks_ch][1],raw[marks_ch][0][0]*100*3e1,'-r', alpha=0.4)
    # plt.plot(raw[sensor_ch][1],raw[sensor_ch][0][0]*100*3e1,'-g', alpha=0.4)
    # plt.legend()
    # =======================================================================

    '''
    -1: erase
    -2: start drawing
    -3: end drawing
    '''
    # create events that correspond to the "start drawing" signal
    events_times_new=np.where(xnew==-2)[0]
    vvv=np.where(events_times_new[1:]-events_times_new[:-1]==1)[0]
    events_times_new=np.delete(events_times_new, vvv)
    events_new=events.copy()[:-1]

    # # Making Epochs object
    events_new[:,0]=events_times_new[:]+adding_in_ts
    raw_marks=raw.copy().drop_channels(raw.ch_names[:-2])
    raw_data=raw.copy().drop_channels(raw.ch_names[-2:])

    # MAKE EPOCHS OF ALL CHANNELS BASED ON "START DRAWING SIGNAL"
    epochs=mne.Epochs(raw,events=events_new, event_id=0, tmin=-2,tmax=7,preload=True, baseline=None,
                    reject_by_annotation=False)
    # MAKE EPOCHS OF EEG CHANNELS BASED ON "START DRAWING SIGNAL"
    epochs_data=mne.Epochs(raw_data,events=events_new, event_id=0, tmin=-2,tmax=7,preload=True, baseline=None, 
                    reject_by_annotation=False)
    # MAKE EPOCHS OF MARKS CHANNELS BASED ON "START DRAWING SIGNAL"
    epochs_marks=mne.Epochs(raw_marks,events=events_new, event_id=0, tmin=-2,tmax=7,preload=True, baseline=None,
                    reject_by_annotation=False)

    num=len(epochs.ch_names)
    # save epochs
    os.makedirs(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}', exist_ok=True)
    epochs_data.save(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/epochs-long-epo.fif', overwrite = True)
    epochs_marks.save(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/epochs-marks-long-epo.fif', overwrite = True)
    np.savetxt(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/events_array_eeg_data.csv',events_new)

    # save data from the tablet as Epochs object
    ch_names=epochs.ch_names
    info = mne.create_info(ch_names=['x', 'y'],
                        ch_types=['eeg'] * 2,
                        sfreq=raw.info['sfreq'])
    data=np.vstack([xnew,ynew])
    simulated_raw = mne.io.RawArray(data, info)

    events_times_new=np.where(xnew==-2)[0]
    vvv=np.where(events_times_new[1:]-events_times_new[:-1]==1)[0]
    events_times_new=np.delete(events_times_new, vvv)
    events_new=events.copy()[:-1]
    events_new[:,0]=events_times_new
    
    # MAKE EPOCHS OF TRAJECTORIES BASED ON "START DRAWING SIGNAL"
    epochs_handwriting=mne.Epochs(simulated_raw,events=events_new, event_id=0, tmin=-2,tmax=4,preload=True, baseline=None,
                    reject_by_annotation=False, verbose=True)
    # save these events
    epochs_handwriting.save(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/epochs_handwriting_tablet-epo.fif', overwrite=True)
    np.savetxt(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/events_array_tablet_data.csv',events_new)

    # separately extract time points when drawing started and create event from them (to further drop epochs based on trial duration)
    events_times_2=np.where(xnew==-2)[0]
    vvv=np.where(events_times_2[1:]-events_times_2[:-1]==1)[0]
    events_times_2=np.delete(events_times_2, vvv)
    events_new2=events.copy()[:-1]
    events_new2[:,0]=events_times_2
    events_new2[:, 2] = -2
    # separately extract time points when drawing ended and create event from them (to further drop epochs based on trial duration)
    events_times_3=np.where(xnew==-3)[0]
    vvv=np.where(events_times_3[1:]-events_times_3[:-1]==1)[0]
    events_times_3=np.delete(events_times_3, vvv)
    events_new3=events.copy()[:-1]
    events_new3[:,0]=events_times_3
    events_new3[:, 2] = -3

    # save these events
    events_pd = pd.DataFrame(np.concatenate([events_new2, events_new3])).sort_values(by = 0).values
    np.savetxt(f'{path_out}/hand_writing/{subj}/epochs/Rec_{code}/events_2-3_tablet_data.csv',events_pd)

In [ ]:
# PLOT TO CHECK EVENTS LOCATION
%matplotlib qt
const = 0
plt.plot(times-times[0]+adding_in_secs+const, x,'.-b', alpha=0.4)
plt.scatter(times[np.where(x==-1)[0]]-times[0]+adding_in_secs+const, xnew[np.where(x==-1)[0]], color = 'black', label = 'erase')

#plt.scatter(timesnew[np.where(xnew==-1)[0]]-timesnew[0]+adding_in_secs, xnew[np.where(xnew==-1)[0]])
plt.scatter(timesnew[np.where(xnew==-2)[0]]-timesnew[0]+adding_in_secs+const, xnew[np.where(xnew==-2)[0]], color = 'blue', label = 'draw start')


plt.plot(raw[marks_ch][1],raw[marks_ch][0][0]*100*3e1,'-r', alpha=0.4)
plt.plot(raw[sensor_ch][1],raw[sensor_ch][0][0]*100*3e1,'-g', alpha=0.4)
plt.legend()